# Procure Safegraph Pattersn on Amazon EC2

This notebook is intended to speed up extracting the patterns data by taking the processing to the data. The Safeagraph Data resides in a Simple Storage Service (S3) bucet in the the US-East-2 region. Therefore, the most efficient way to get the data we need...aside from recruiting a true _big data_ infrastructure stack, is to run an Amazon Elastic Compute Cloud (EC2) instance with Python, Jupyter Notebook and a copy of this repository, and use this execution environment to extract just the data we need.

In [1]:
from pathlib import Path

from sg_data import SafegraphClient

In [2]:
dir_prj = Path('./').absolute().parent

dir_data = dir_prj/'data'

dir_raw = dir_data/'raw'

For the notebook to run, you will need to populate the variables below with the information you get from your own account credentials provided by Safegraph for data access.

In [3]:
AWS_KEY = 'AO1L8A67NB5QUYLQOYLU'
AWS_SECRET = 'iECNEV7kUMDtAv701meTrwtPFVysh6YaBecDEehY'

sg_poi = 'sg:af471021a929414cbf69854e6f8f1b0c'  # white pass

Using the credentials provided above, a Safegraph Client object instance can be created. This object is part of the Safegraph Data Utilities provided with this package.

In [4]:
sg = SafegraphClient(access_key=AWS_KEY, secret_key=AWS_SECRET)

sg

One of the functions of the Safegraph Client provides introspection, the ability to interrogate the available resources to discover what is available as a Pandas DataFrame.

In [5]:
ptrns_df = sg.content_dataframe

ptrns_df.head()

,source_path,year,month,resource_type,standardized_path
0,monthly-patterns/brand_info_backfill/2020/12/1...,2018,1,brand_info,monthly-patterns/brand_info/2018/01/brand_info...
1,monthly-patterns/brand_info_backfill/2020/12/1...,2018,2,brand_info,monthly-patterns/brand_info/2018/02/brand_info...
2,monthly-patterns/brand_info_backfill/2020/12/1...,2018,3,brand_info,monthly-patterns/brand_info/2018/03/brand_info...
3,monthly-patterns/brand_info_backfill/2020/12/1...,2018,4,brand_info,monthly-patterns/brand_info/2018/04/brand_info...
4,monthly-patterns/brand_info_backfill/2020/12/1...,2018,5,brand_info,monthly-patterns/brand_info/2018/05/brand_info...


This DataFrame can be filtered and organized to discover what is available. In this case we are interested in what `patterns` data is available, and to more easily see the range available, we are sorting by month and year.

In [6]:
ptrns_df = ptrns_df[ptrns_df['resource_type'] == 'patterns'].sort_values(['year', 'month']).copy()

Finally, although we are going to retrieve three years worth of data, since interested only in ski season, we are filtering to November through April. Although not instantanious, when I performed the same data pull on a local machine in my office, this process took over six hours, so this is a dramatic improvement.

In [7]:
row = ptrns_df.iloc[0]
src = row.source_path

In [8]:
# https://aws.amazon.com/blogs/storage/querying-data-without-servers-or-databases-using-amazon-s3-select/
# https://docs.aws.amazon.com/AmazonS3/latest/API/API_SelectObjectContent.html
resp = sg.s3.select_object_content(
    Bucket=sg.bucket,
    Key=src,
    ExpressionType='SQL',
    Expression=f"SELECT * FROM s3object s where s.\"safegraph_place_id\" = '{sg_poi}'",
    InputSerialization = {'CSV': {"FileHeaderInfo": "Use"}, 'CompressionType': 'GZIP'},
    OutputSerialization = {'CSV': {}},
)

for event in resp['Payload']:
    if 'Records' in event:
        records = event['Records']['Payload'].decode('utf-8')
        print(records)
    elif 'Stats' in event:
        statsDetails = event['Stats']['Details']

ClientError: An error occurred (AccessDenied) when calling the SelectObjectContent operation: Access Denied